In [1]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [2]:
import torch
print(torch.version.cuda)        # PyTorch CUDA version
print(torch.cuda.is_available()) # Should be True
print(torch.cuda.get_device_name(0))


12.1
True
NVIDIA GeForce RTX 3050 Laptop GPU


In [10]:
from ultralytics import YOLO

# Load last checkpoint
model = YOLO(r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\Hard Hats.v6-resized640_noaugmentation-rf-detr.yolov5pytorch\runs\detect\train6\weights\last.pt")

# Resume training
model.train(
    data=r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\Hard Hats.v6-resized640_noaugmentation-rf-detr.yolov5pytorch\data.yaml",
    resume=True,
    imgsz=640,   # can increase to 800/960 if GPU allows for higher accuracy
    epochs=30       
)


New https://pypi.org/project/ultralytics/8.3.193 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.187  Python-3.11.0 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=1, bgr=0.0, box=7.5, cache=disk, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\Hard Hats.v6-resized640_noaugmentation-rf-detr.yolov5pytorch\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=D:\Sain

KeyboardInterrupt: 

In [1]:
import cv2
from ultralytics import YOLO
import pygame
import time
import os

# Initialize pygame mixer
pygame.mixer.init()

# Load alert sound
alert_sound = r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\Hard Hats.v6-resized640_noaugmentation-rf-detr.yolov5pytorch\nohelmetalert.mp3"

# Load model
model = YOLO(r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\Hard Hats.v6-resized640_noaugmentation-rf-detr.yolov5pytorch\runs\detect\train5\weights\last.pt")

# Input / Output video
video_path = r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\Hard Hats.v6-resized640_noaugmentation-rf-detr.yolov5pytorch\helmet2.mp4"
output_path = r"D:\Sainath_Backup\Desktop\UCHIT TECHNOLOGIES\VMS\Hard Hats.v6-resized640_noaugmentation-rf-detr.yolov5pytorch\helmet_detected_output1.mp4"

# Folder for saving "NO-Hardhat" screenshots
screenshot_dir = os.path.join(os.path.dirname(output_path), "nohardhatsamples")
os.makedirs(screenshot_dir, exist_ok=True)

# Video setup
cap = cv2.VideoCapture(video_path)
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, int(cap.get(5)), (int(cap.get(3)), int(cap.get(4))))

frame_no = 0
screenshot_count = 0

# Track individual persons without hardhats
class PersonTracker:
    def __init__(self):
        self.persons = {}  # person_id: {'max_conf': float, 'best_frame': frame, 'alerted': bool, 'last_seen': frame_no}
        self.next_id = 1
        self.iou_threshold = 0.3  # For matching detections across frames
        self.max_frames_missing = 30  # Remove person if not seen for 30 frames
    
    def calculate_iou(self, box1, box2):
        """Calculate Intersection over Union (IoU) of two bounding boxes"""
        x1_inter = max(box1[0], box2[0])
        y1_inter = max(box1[1], box2[1])
        x2_inter = min(box1[2], box2[2])
        y2_inter = min(box1[3], box2[3])
        
        if x2_inter <= x1_inter or y2_inter <= y1_inter:
            return 0.0
        
        inter_area = (x2_inter - x1_inter) * (y2_inter - y1_inter)
        box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
        box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
        union_area = box1_area + box2_area - inter_area
        
        return inter_area / union_area if union_area > 0 else 0
    
    def update(self, no_hardhat_detections, frame, current_frame_no):
        """Update tracker with new detections"""
        alerts_triggered = []
        
        # Mark all persons as not seen this frame
        for person_id in self.persons:
            self.persons[person_id]['seen_this_frame'] = False
        
        # Match new detections with existing persons
        for detection in no_hardhat_detections:
            x1, y1, x2, y2, conf = detection
            current_box = [x1, y1, x2, y2]
            
            # Find best matching existing person
            best_match_id = None
            best_iou = 0
            
            for person_id, person_data in self.persons.items():
                if 'last_box' in person_data:
                    iou = self.calculate_iou(current_box, person_data['last_box'])
                    if iou > best_iou and iou > self.iou_threshold:
                        best_iou = iou
                        best_match_id = person_id
            
            if best_match_id is not None:
                # Update existing person
                person = self.persons[best_match_id]
                person['last_seen'] = current_frame_no
                person['last_box'] = current_box
                person['seen_this_frame'] = True
                
                # Check if this is a new highest confidence
                if conf > person['max_conf']:
                    person['max_conf'] = conf
                    person['best_frame'] = frame.copy()
                    
                    # Trigger alert if not already alerted for this person
                    if not person['alerted']:
                        person['alerted'] = True
                        alerts_triggered.append((best_match_id, conf, frame.copy()))
            else:
                # Create new person
                person_id = self.next_id
                self.next_id += 1
                
                self.persons[person_id] = {
                    'max_conf': conf,
                    'best_frame': frame.copy(),
                    'alerted': True,  # Alert immediately for new detection
                    'last_seen': current_frame_no,
                    'last_box': current_box,
                    'seen_this_frame': True
                }
                
                alerts_triggered.append((person_id, conf, frame.copy()))
        
        # Remove persons not seen for too long
        to_remove = []
        for person_id, person_data in self.persons.items():
            if not person_data['seen_this_frame']:
                if current_frame_no - person_data['last_seen'] > self.max_frames_missing:
                    to_remove.append(person_id)
        
        for person_id in to_remove:
            del self.persons[person_id]
        
        return alerts_triggered

# Initialize tracker
tracker = PersonTracker()

# Track last alert time to prevent audio overlap
last_alert_time = 0
min_alert_interval = 3.0  # Minimum 3 seconds between alerts (matching your original cooldown)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO detection
    results = model(frame)
    
    no_hardhat_detections = []
    current_time = time.time()

    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls[0])
            label = model.names[cls_id]
            conf = float(box.conf[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            
            color = (0, 255, 0) if label == "Hardhat" else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            
            # Collect NO-Hardhat detections for tracking
            if label == "NO-Hardhat":
                no_hardhat_detections.append((x1, y1, x2, y2, conf))

    # Update tracker and get alerts
    alerts = tracker.update(no_hardhat_detections, frame, frame_no)
    
    # Process alerts
    for person_id, conf, alert_frame in alerts:
        # Play alert sound (with timing control to prevent overlap)
        if current_time - last_alert_time >= min_alert_interval:
            try:
                pygame.mixer.music.load(alert_sound)
                pygame.mixer.music.play()
                last_alert_time = current_time
                print(f"🔊 Alert sound played for Person {person_id}")
            except pygame.error as e:
                print(f"Audio error: {e}")

        # Save screenshot of the detection with highest confidence
        screenshot_count += 1
        screenshot_path = os.path.join(screenshot_dir, f"nohardhat_person{person_id}_conf{conf:.2f}_{screenshot_count}.jpg")
        cv2.imwrite(screenshot_path, alert_frame)
        print(f"⚠️ Person {person_id} without hardhat detected! Confidence: {conf:.2f} - Screenshot saved: {screenshot_path}")

    # Add tracking info to display
    if tracker.persons:
        y_offset = 30
        for person_id, person_data in tracker.persons.items():
            info_text = f"Person {person_id}: Max Conf {person_data['max_conf']:.2f}"
            cv2.putText(frame, info_text, (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            y_offset += 25

    # Write frame to output video
    out.write(frame)
    
    # Display frame
    cv2.imshow("Helmet Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    frame_no += 1

# Save final screenshots for all tracked persons with their highest confidence frames
print("\n📸 Saving final best screenshots for all tracked persons:")
for person_id, person_data in tracker.persons.items():
    final_screenshot_path = os.path.join(screenshot_dir, f"BEST_nohardhat_person{person_id}_conf{person_data['max_conf']:.2f}.jpg")
    cv2.imwrite(final_screenshot_path, person_data['best_frame'])
    print(f"💾 Person {person_id} best screenshot (conf: {person_data['max_conf']:.2f}) saved: {final_screenshot_path}")

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"\n✅ Detection complete. Video saved at: {output_path}")
print(f"📸 Screenshots saved in: {screenshot_dir}")
print(f"👥 Total persons without hardhat tracked: {len(tracker.persons)}")
print(f"🔊 Total alerts triggered: {screenshot_count}")

c:\Users\nikam\AppData\Local\Programs\Python\Python311\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html

0: 512x288 1 Hardhat, 209.0ms
Speed: 15.3ms preprocess, 209.0ms inference, 1211.6ms postprocess per image at shape (1, 3, 512, 288)

0: 512x288 1 Hardhat, 21.9ms
Speed: 3.3ms preprocess, 21.9ms inference, 3.7ms postprocess per image at shape (1, 3, 512, 288)

0: 512x288 1 Hardhat, 23.1ms
Speed: 3.1ms preprocess, 23.1ms inference, 3.2ms postprocess per image at shape (1, 3, 512, 288)

0: 512x288 1 Hardhat, 21.9ms
Speed: 3.2ms preprocess, 21.9ms inference, 2.3ms postprocess per image at shape (1, 3, 512, 288)

0: 512x288 1 Hardhat, 22.2ms
Speed: 2.8ms preprocess, 22.2ms inference, 2.3ms postprocess per image at shape (1, 3, 512, 288)

0: 512x288 1 Hardhat, 22.1ms
Speed: 2.8ms preprocess, 22.1ms inference, 3.3ms postprocess per image at shape (1, 3, 512, 288)

0: 512x288 1 Hardhat, 22.2ms
Speed: 3.5ms preprocess, 22.2ms inference, 3.0ms postprocess per image at shape (1, 3, 51